In [2]:
import requests
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np

In [3]:
import json
import pickle as pkl
from scipy import stats
def load_pkl(path):
    data=pkl.load(open(path,'rb'))
    return data

def load_json(path):
    data=json.load(open(path,'r'))
    return data

import pandas as pd
from collections import defaultdict

In [17]:
import os
import base64
import random

In [ ]:
path=os.path.abspath('../..')
print (path)
save_dir='prepare_submission/converted_results'
p2_data=load_json(os.path.join(path,'data/data_clean/split_data/test.json'))
test_id_to_row={}
for i,row in enumerate(p2_data):
    test_id_to_row[i]=row

In [43]:

llm_name='qwen'
mllm_name='qwen'

#llm_name='qwen'
#mllm_name='llava'

#llm_name='gemma'
#mllm_name='gemma'

#llm_name="gemini-2.0-flash-001"
#mllm_name="gemini-2.0-flash-001"

save_num='12'
threshold=9

pred_file=load_pkl(os.path.join(path,'fc_detailed_results','_'.join([llm_name,mllm_name]),save_num+'.pkl'))
print (len(pred_file))

save_name=os.path.join(path,save_dir,'_'.join([llm_name,mllm_name,str(save_num)])+'.json')

352


In [44]:
total=[]

for i,row in enumerate(p2_data):
    req_id=i
    result=pred_file[req_id]
    #if i>0:
    #    break
    """
    if i>0:
        break
    else:
        print (result.keys())
        print (result['QA_info'])
        print (result['evidence'])
    """
    entry={
        'id':i,
        'questions':[ent['question'] for ent in result['QA_info']],
        'justification':result['justification'],
        'verdict':result['verdict']
    }
    evidence=[]
    for evid in result['evidence']:
        evid_text=evid['text']
        evid_img=evid['images']
        images=[]
        for image_path in evid_img:
            with open(image_path, 'rb') as image_file:
                image_data = image_file.read()
                base64_encoded = base64.b64encode(image_data).decode('utf-8')
                images.append(base64_encoded)
        evidence.append({
            'text':evid_text,
            'images':images
        })
    entry['evidence']=evidence
    total.append(entry)

In [45]:
json.dump(total,open(save_name,'w'))